In [23]:
from wiki_labels import qid_lab_get, find_qid, trie
from text_utils import get_provenance
import json
import os
import numpy as np

BASE_DIR, DB_DIR = '/projekti/mondoAPI', 'db'

types = json.load(open(os.path.join(BASE_DIR, DB_DIR, 'types.json'), 'r'))
types_a = np.array([tuple([b if b else '' for b in a]) if len(a) == 4 else ('', '', '', '') for a in types],
                   [('tip', 'U3'), ('lang', 'U2'), ('script', 'U4'), ('cc', 'U2')])
t2i = {'_'.join(v):i for i, v in enumerate(types_a)}


In [ ]:
male = ['Q8441', 'Q212878', 'Q17051299', 'Q12826225', 'Q2366336', 'Q1092777', 'Q854905', 'Q435940', 'Q177053', 'Q1937846', 'Q2747456', 'Q12097', 'Q749212', 'Q209690', 'Q102083', 'Q166382', 'Q165503']
female = ['Q467', 'Q188830', 'Q16963317', 'Q901407', 'Q104154414', 'Q13359947', 'Q1951543', 'Q313549', 'Q901407', 'Q863048', 'Q16511993', 'Q1378024', 'Q13359947']
unisex = ['Q12540025', 'Q678928', 'Q48942']

In [ ]:
fi = open('/backup/wikidata/wiki_gender_label.tsv')
fi.readline()
gen = {}
for i in fi:
    g, wid, lab, lang = i.strip('\n\r').split('\t')
    if lab in gen and gen[lab] != g and len(gen[lab]) == 1:
        gen[lab] = 'mf'
    else:
        gen[lab] = g

In [69]:
honor = {}
genders = ((male, 'm'), (female, 'f'), (unisex, 'mf'))
for ds, gender in genders:
    for i in ds:
        labs = qid_lab_get(int(i[1:]), include_alt=True)
        if i == 'Q177053':
            print(labs)
        for lab in labs:
            for lang in labs[lab]:
                provs = get_provenance(lab, lang, no_countries=True)
                for prov in provs:
                    k = (lab, prov)
                    if lab in gen and (gen[lab] == 'mf' or gen[lab] != gender):
                        g = 'mf'
                    elif lab in gen:
                        g = gen[lab]
                    else:
                        g = gender
                    if k not in honor:
                        honor[k] = g
                    elif honor[k] != g and len(honor[k]) == 1:
                        honor[k] = 'mf'
                       
len(honor)

{'mr': {'en'}, 'g.': {'tl'}, 'sr.': {'es'}, 'pan': {'pl', 'cs'}, 'mr.': {'en', 'nl', 'ko'}, 'bay': {'tr'}, 'úr': {'hu'}, 'saks': {'et'}, 'sig.': {'it'}, 's-ro': {'eo'}, 'pán': {'sk'}, 'knj.': {'hsb'}, 'domn': {'ro'}, 'herr': {'da', 'sv', 'de'}, 'jaun': {'eu'}, 'señor': {'es'}, 'sciur': {'lmo'}, 'knjez': {'hsb'}, 'budni': {'pam'}, 'herre': {'nb'}, 'herra': {'fi'}, 'ginoo': {'tl'}, 'signor': {'it'}, 'maighstir': {'gd'}, 'meneer': {'nl'}, 'mister': {'en'}, 'mudane': {'so'}, 'domine': {'la'}, 'härra': {'et'}, 'sinjoro': {'eo'}, 'budning': {'pam'}, 'ginoong': {'tl'}, 'monsieur': {'fr'}, 'gospodin': {'sh'}, 'مستر': {'ar'}, '미스터': {'ko'}, 'خواجه': {'fa'}, 'мистер': {'mk', 'ru'}, 'містер': {'uk'}, 'спадар': {'be-tarask', 'be'}, 'сударь': {'ru'}, 'ミスター': {'ja'}, 'господин': {'mk', 'ru'}}


1961

In [70]:
# few corrections
honor[('先生', 'zh_Hani_')] = 'm'
honor[('さん', 'ja_Kana_')] = 'mf'
honor[('さま', 'ja_Kana_')] = 'mf'
honor[('様', 'ja_Hira_')] = 'mf'

honor[('mstr', 'en_Latn_')] = 'm'
honor[('mstr.', 'en_Latn_')] = 'm'
honor[('master', 'en_Latn_')] = 'm'
honor[('datuk', 'ms_Latn_')] = 'm'
honor[('datuk.', 'ms_Latn_')] = 'm'
honor[('datuk seri', 'ms_Latn_')] = 'm'
honor[('datuk seri', 'ms_Latn_')] = 'm'
honor[('dato', 'ms_Latn_')] = 'm'
honor[('datu', 'ms_Latn_')] = 'm'
honor[('dato.', 'ms_Latn_')] = 'm'
honor[('datu.', 'ms_Latn_')] = 'm'
honor[('tan sri', 'ms_Latn_')] = 'm'
honor[('puan sri', 'ms_Latn_')] = 'f'
honor[('puan sri.', 'ms_Latn_')] = 'f'
honor[('datin', 'ms_Latn_')] = 'f'
honor[('datin.', 'ms_Latn_')] = 'f'
honor[('datin seri', 'ms_Latn_')] = 'f'
honor[('datin seri.', 'ms_Latn_')] = 'f'
honor[('seri.', 'ms_Latn_')] = 'f'

honor[('mevrouw', 'nl_Latn_')] = 'f'
honor[('mw.', 'nl_Latn_')] = 'f'


for n in ('เด็กหญิง', 'นางสาว', 'ด.ญ.', 'นาง', 'คุณหญิง', 'นารี'):
    honor[(n, 'th_Thai_')] = 'f'
for n in ('นาย', 'เด็กชาย', 'มร.', ):
    honor[(n, 'th_Thai_')] = 'm'
honor[('คุณ', 'th_Thai_')] = 'mf'

In [72]:
[(l, p, g) for (l, p), g in honor.items() if l == 'ด.ญ.'] 

[('ด.ญ.', 'th_Thai', 'f')]

In [80]:
# to Oracle
import cx_Oracle
tls = 'db202201301230_high'
connection = cx_Oracle.connect(user='mondo', password=open('orapsw.txt').read(),dsn=tls,encoding="UTF-8")
cur = connection.cursor()
cur.execute('drop table temp_honorific purge')
cur.execute('create table temp_honorific (tag varchar2(50), n varchar2(100))')
for (l, p), g in honor.items():
    tit = "title"
    if g == "mf":
        pass
    elif g == 'm':
        tit += '1'
    elif g == 'f':
        tit += '2'
    else:
        raise NotImplementedError
    tit += '_'+(p.strip('_'))
    cur.execute('insert into temp_honorific values (:1, :2)', (tit, l))
connection.commit()    